In [1]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split, cross_val_score
# from tpot.config.classifier_nn import classifier_config_nn

from sklearn.pipeline import make_pipeline
# from tpot.config import classifier_config_dict_light
from tpot.config import classifier_config_dict
from sklearn.neighbors import KNeighborsClassifier


import pandas as pd
import numpy as np
import os
import glob

import time

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
pd.options.mode.chained_assignment = None

Using TensorFlow backend.


In [2]:
# personal_config = classifier_config_dict_light
personal_config = classifier_config_dict
personal_config['tpot.builtins.SimpleAutoencoder'] = {
    'encoding_dim': [10, 50, 100, 500],
    'activation': ['relu'],
    'optimizer': ['adadelta', "SGD", "Adam", "Adamax", "Nadam"],
    'loss':['binary_crossentropy', 'hinge', 'mean_squared_error', 'mean_absolute_error'],
    'epochs':[50, 100],
    'batch_size':[10, 15]
}

# personal_config['tpot.builtins.SimpleAutoencoder'] = {
#     'regularizer' :['regularizers.l1(10e-5)', 'regularizers.l2(10e-5)', 'regularizers.l1_l2(10e-5, 10e-5)'],
#     'encoding_dim': [40],
#     'activation': ['relu'],
#     'optimizer': ['adadelta', "SGD"],
#     'loss':['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error',
#        'squared_hinge', 'hinge', 'logcosh', 'binary_crossentropy', 'kullback_leibler_divergence', 'cosine_proximity', 'poisson'],
#     'epochs':[100],
#     'batch_size':[200]
# }

In [3]:
#new data
new_data = pd.read_csv("processed_aml_data.csv")
X = new_data.iloc[:, 1:5051]
y = new_data.iloc[:, 5051]

In [4]:
#convert X and y to numpy
X = X.values
y = y.values



In [5]:
#split data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(X, y, train_size = 0.8, test_size = 0.20, random_state = 42)
# x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
# x_val = x_val.reshape((len(x_val)), np.prod(x_val.shape[1:]))

In [6]:
#x_train.shape

In [7]:
#y_train.shape

In [8]:
from tpot.builtins import SimpleAutoencoder

In [12]:
# #sae = SimpleAutoencoder(encoding_dim=10, 
#                         activation='relu', 
#                         optimizer='adadelta', 
#                         loss='binary_crossentropy', 
#                         epochs=50, batch_size=10)

In [13]:
#sae.fit(x_train, y_train)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


SimpleAutoencoder(activation='relu', batch_size=10, encoding_dim=10,
         epochs=50, loss='binary_crossentropy', optimizer='adadelta',
         random_state=42)

In [10]:
gens = 1
pop_size = 10
template = "SimpleAutoencoder-RandomForestClassifier"

In [11]:
tpot = TPOTClassifier(generations=gens, config_dict=personal_config,
                        population_size=pop_size, verbosity=3,
                        template = template)

In [13]:
start = time.time()
tpot.fit(x_train, y_train)
end = time.time()
run_time = end - start
print(run_time)

31 operators have been imported by TPOT.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


Generation 1 - Current Pareto front scores:
-2	0.7178571428571429	RandomForestClassifier(SimpleAutoencoder(input_matrix, SimpleAutoencoder__activation=relu, SimpleAutoencoder__batch_size=10, SimpleAutoencoder__encoding_dim=10, SimpleAutoencoder__epochs=50, SimpleAutoencoder__loss=hinge, SimpleAutoencoder__optimizer=SGD), RandomForestClassifier__bootstrap=False, RandomForestClassifier__criterion=gini, RandomForestClassifier__max_features=0.55, RandomForestClassifier__min_samples_leaf=9, RandomForestClassifier__min_samples_split=4, RandomForestClassifier__n_estimators=100)

912.0336120128632


In [14]:
final_pipeline = tpot.fitted_pipeline_

In [15]:
training_acc = tpot.score(x_train, y_train)

In [16]:
testing_acc = tpot.score(x_val, y_val)

In [21]:
results_dict = {'generations': [gens], 'population_size': [pop_size],
                'training_acc': [training_acc], 'testing_acc': [testing_acc], 'pipeline': [final_pipeline]}
simple_aml_results_df = pd.DataFrame(data=results_dict)

In [22]:
simple_aml_results_df


,generations,population_size,training_acc,testing_acc,pipeline
0,1,10,0.833333,0.545455,"Pipeline(memory=None,\n steps=[('simpleaut..."


In [25]:
simple_aml_results_df.to_csv('simple_aml_results.csv',index=False)